In [2]:
ugr16_column_name = 'te', 'td', 'sa', 'da', 'sp', 'dp', 'pr', 'flg', 'fwd', 'stos', 'pkt', 'byt', 'label'
stan_column_name =  'teT', 'teDelta', 'log_byt', 'log_pkt', 'td', 'sa', 'da', 'sp', 'dp', 'pr'
data_path = '/home/ubuntu/Documents/data/' #path for all data files
prt_path = data_path + 'stan/partitioned_data/'
raw_file = data_path + 'ugr16/april_week3_day1.csv' #source ugr16 file
pre_file = data_path + 'stan/preprocessed_data/april_week3_day1_PREPROCESSED.csv'

Data Injest

Creates data frame of raw netwflow data and sorts it in the formate stan.NetflowFormatTranformer.transfer expects


In [2]:
from pyspark.sql.functions import log, split, when
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local").appName("stan_eval").getOrCreate()

local_ip = '42.219'
df_raw = sc.read.csv(raw_file)
df_raw = df_raw.toDF(*ugr16_column_name)

df_working = df_raw.withColumn('log_byt', log(df_raw.byt))
df_working = df_working.withColumn('log_pkt', log(df_working.pkt))


# pull minutes hours and seconds from timestamp into a single number
df_working = df_working.withColumn('time_text', split(df_working.te, " ").getItem(1))

df_working = df_working.withColumn('seconds', \
                                   split(df_working.time_text, ':')[0] * 3600 + \
                                   split(df_working.time_text, ':')[1] * 60 + \
                                   split(df_working.time_text, ':')[2] )

df_working = df_working.withColumn('local_ip', \
                      when(df_working.sa.startswith(local_ip), df_working.sa)\
                      .when(df_working.da.startswith(local_ip), df_working.da)\
                      .otherwise(0))

#df_working = df_working.withColumn('teDelta', ) #pull minute from timestamp plus
df_working.show(20)

bash: switchml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `switchml'
bash: _moduleraw: line 1: syntax error: unexpected end of file
bash: error importing function definition for `_moduleraw'
bash: _moduleraw: line 1: syntax error: unexpected end of file
bash: error importing function definition for `_moduleraw'
bash: switchml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `switchml'
22/06/23 08:04:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+-------------------+------+--------------+---------------+-----+-----+---+------+---+----+---+-------+----------+------------------+------------------+---------+-------+--------------+
|                 te|    td|            sa|             da|   sp|   dp| pr|   flg|fwd|stos|pkt|    byt|     label|           log_byt|           log_pkt|time_text|seconds|      local_ip|
+-------------------+------+--------------+---------------+-----+-----+---+------+---+----+---+-------+----------+------------------+------------------+---------+-------+--------------+
|2016-04-11 00:01:34| 5.016| 74.159.162.35|  42.219.155.26|24331|  443|TCP|.AP.S.|  0|   0|  9|   1807|background| 7.499423290592229|2.1972245773362196| 00:01:34|   94.0| 42.219.155.26|
|2016-04-11 00:01:34| 5.400| 165.13.187.32| 42.219.156.211|47976|  443|TCP|.AP.S.|  0|   0| 10|   1662|background| 7.415776975415394| 2.302585092994046| 00:01:34|   94.0|42.219.156.211|
|2016-04-11 00:01:34| 5.440| 165.13.187.32| 42.219.156.211|47977|  443

In [3]:
from pyspark.sql.functions import lag, isnull, floor
stan_window = Window.partitionBy('local_ip').orderBy("seconds")

df_working = df_working.withColumn('prev_seconds', lag(df_working.seconds).over(stan_window))

df_working = df_working.withColumn('teDelta', \
                                   when(isnull(df_working.seconds - df_working.prev_seconds), 0)\
                                   .otherwise(df_working.seconds - df_working.prev_seconds))

df_out = df_working.select('seconds', 'teDelta', 'log_byt', 'byt', 'log_pkt', 'pkt', 'td', 'sa', 'da', 'sp', 'dp', 'pr', 'local_ip')

df_out = df_out.withColumn("this_ip", df_out.local_ip)
df_out = df_out.withColumn("teT", floor((df_out.seconds / 3600)))

In [8]:
df_out.show(200)

+-------+-------+------------------+---+------------------+---+-----+---------------+--------------+-----+-----+----+--------------+--------------+---+
|seconds|teDelta|           log_byt|byt|           log_pkt|pkt|   td|             sa|            da|   sp|   dp|  pr|      local_ip|       this_ip|teT|
+-------+-------+------------------+---+------------------+---+-----+---------------+--------------+-----+-----+----+--------------+--------------+---+
|  389.0|    0.0|6.0844994130751715|439|               0.0|  1|0.000|176.116.195.149|42.219.145.100| 5185| 5074| UDP|42.219.145.100|42.219.145.100|  0|
|  648.0|  259.0| 4.174387269895637| 65|               0.0|  1|0.000|  56.207.76.118|42.219.145.100|  999|   53| UDP|42.219.145.100|42.219.145.100|  0|
|  775.0|  127.0| 3.332204510175204| 28|               0.0|  1|0.000|  69.81.144.249|42.219.145.100|41347|   19| UDP|42.219.145.100|42.219.145.100|  0|
|  960.0|  185.0|3.6888794541139363| 40|               0.0|  1|0.000|   246.183.3.40|42.

In [4]:
df_out.write.option('header', True).mode("overwrite").partitionBy("local_ip").csv(prt_path)

In [5]:
from stannetflow import preprocess as p

p.prepare_standata(train_folder=prt_path, train_output=pre_file)

making train for:
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.156.96/part-00115-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.152.255/part-00071-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.154.168/part-00066-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.154.233/part-00079-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.146.211/part-00183-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.148.31/part-00092-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Documents/data/stan/partitioned_data/local_ip=42.219.153.6/part-00081-ed39d647-f073-40e6-a649-946ef5405d13.c000.csv
user: /home/ubuntu/Document

KeyboardInterrupt: 